In [49]:
import pandas as pd

In [50]:
four= pd.DataFrame.from_csv("C:\Users\Debmalya\Documents\GitHub\economic_base\data\\4_digit.csv", header= 0, index_col= None)
#len(four)
four.dtypes

GEO.id                  object
GEO.id2                float64
GEO.display-label       object
NAICS.id                object
NAICS.display-label     object
OPTAX.id                object
OPTAX.display-label     object
YEAR.id                  int64
ESTAB                   object
RCPTOT                  object
PAYANN                  object
EMP                     object
NESTAB                  object
NRCPTOT                 object
dtype: object

In [56]:
four= four[(four["OPTAX.id"] != "10") & (four["OPTAX.id"] != "20") & (four["OPTAX.id"] != "T") & (four["OPTAX.id"] != "Y")]
#len(four) # the above to select specific data

In [57]:
US= four[four["GEO.display-label"]=="United States"]
US= US[["NAICS.id", "EMP"]] # remember during column subset two brackets, like getitem
US.rename(columns={'EMP': 'US_EMP'}, inplace=True) # rename columns
#US.head()

In [58]:
KINGS= four[four["GEO.display-label"]== "Kings County, California"]
KINGS= KINGS[["NAICS.id", "EMP"]]
KINGS.rename(columns= {"EMP" : "KINGS_EMP"}, inplace= True) # renaming a column
#KINGS.head()

In [59]:
m=US.merge(KINGS, on= "NAICS.id", how= 'outer') # outer joinn to keep everything
#m

In [60]:
agri_us= pd.DataFrame.from_csv("C:\Users\Debmalya\Documents\GitHub\economic_base\data\\agri_us.csv", header= 0, index_col= None)
#agri_us.head()
agri_us[["NAICS code"]]= agri_us[["NAICS code"]].astype(str)                                     # preparaton for the next line
four_dig = (agri_us["NAICS code"].str.len() == 4)                                           # to find only the four digit codes
us_agri= agri_us.loc[four_dig]                                                                    # subsetting by the list made
us_agri=us_agri.rename(columns = {'Paid employees for pay period including March 12 (number)':'US_EMP',
                                  'NAICS code':'NAICS.id' })
us_agri= us_agri[['NAICS.id', 'US_EMP']]
#us_agri.head()

In [61]:
agri_kings= pd.DataFrame.from_csv("C:\Users\Debmalya\Documents\GitHub\economic_base\data\\agri_kings.csv", 
                                  header= 0, index_col= None)
#agri_kings
agri_kings[["NAICS code"]]= agri_kings[["NAICS code"]].astype(str)
four_digt = (agri_kings["NAICS code"].str.len() == 4)
kings_agri= agri_kings.loc[four_digt]
kings_agri= kings_agri.rename(columns = {'Paid employees for pay period including March 12 (number)':'KINGS_EMP',
                                        'NAICS code':'NAICS.id'})
kings_agri= kings_agri[['NAICS.id', 'KINGS_EMP']]
#kings_agri

In [62]:
agri=us_agri.merge(kings_agri, on= "NAICS.id", how= 'outer')
#agri

In [63]:
occu= [agri,m]
full= pd.concat(occu, ignore_index= True) # concat is rbind
#full

In [64]:
full.replace(to_replace="b", value= '60', inplace=True)  # (20+100)/2
full.replace(to_replace="c", value= '175', inplace=True) # (100+250)/2
full.replace(to_replace="e", value= '375', inplace=True) # (250+500)/2
full.replace(to_replace="f", value= '750', inplace=True) # (500+1000)/2
full.replace(to_replace="g", value= '1750', inplace=True)# (1000+2500)/2
full.replace(to_replace="h", value= '3750', inplace=True)# (2500+5000)/2
full.replace(to_replace="i", value= '7500', inplace=True)# (5000+10000)/2
full.replace(to_replace="j", value= '17500', inplace=True) # (10000+25000)/2
full.replace(to_replace="m", value= '100000', inplace=True)
# and to replace the strings of avg values

In [65]:
# to replace bad data, found from inspection
full.replace(to_replace="19062(r)", value= '19062', inplace=True)

In [66]:
full[["US_EMP"]]= full[["US_EMP"]].astype(float)
full[["KINGS_EMP"]]= full[["KINGS_EMP"]].astype(float)
full.dtypes

NAICS.id      object
US_EMP       float64
KINGS_EMP    float64
dtype: object

In [67]:
USTotal= full.US_EMP.sum()
KINGSTotal= full.KINGS_EMP.sum()
USTotal

108453760.0

In [68]:
KINGSTotal

15263.0

In [69]:
full["KINGS_e"]= full["KINGS_EMP"]/ KINGSTotal
full["KINGS_E"]= full["US_EMP"]/ USTotal
full["Kings_LQ"]= full["KINGS_e"]/ full["KINGS_E"]
full["KINGS_bi"]= ((full["Kings_LQ"]- 1)/full["Kings_LQ"])*full["KINGS_EMP"]

In [70]:
small= full[full.KINGS_bi >=0]

In [71]:
KINGS_bi_sum=small.KINGS_bi.sum()
KINGS_BM=KINGSTotal/KINGS_bi_sum
KINGS_BM

1.4893133800031626